In [1]:
import numpy as np
import lbg_forecast.sps as sps
import lbg_forecast.hyperparameters as hyp
import lbg_forecast.population_model as pop
import lbg_forecast.priors as pr
import lbg_forecast.sfh as sfh
import matplotlib.pyplot as plt

In [2]:
sps_model = sps.initialise_sps_model(neb_em=False, sfh_type=3, zcont=1, imf_type=2, dust_type=0, igm=True)

In [3]:
hyperparameter_mu_bounds, hyperparameter_sigma_max  = hyp.default_hyperparameter_bounds()
prior_parameters = hyp.sample_prior_parameters(1000, hyperparameter_mu_bounds, hyperparameter_sigma_max)

In [4]:
redshift_mass_prior_parameters = pr.setup_redshift_and_mass_priors(z_max=15)

In [5]:
sps_parameters = pop.generate_sps_parameters(1, prior_parameters[np.random.randint(0, 1000),:], redshift_mass_prior_parameters, uniform_redshift_mass=False, uniform_logf=False)
#sps_parameters[0][0] = 1e-5

In [6]:
sps.update_model(sps_model, sps_parameters[0], z_history=False, agebins=sfh.default_agebins())

In [7]:
#spec_fsps = sps.fsps_simulate_sed(sps_model)
#spec_alt = sps.get_sed(sps_model, 'lsst')

In [8]:
photo_fsps = sps.fsps_get_magnitudes(sps_model, 'lsst')
photo_alt = sps.get_magnitudes(sps_model, 'lsst')

In [9]:
photo_fsps-photo_alt

array([28.3051521 , 27.73699472, 27.16696839, 26.75054286, 26.44130809,
       26.19441612])

In [10]:
photo_fsps

array([26.92960438, 26.98655488, 27.04671199, 27.10325149, 27.20786438,
       27.01326373])

In [11]:
photo_alt

array([-1.37554772, -0.75043984, -0.1202564 ,  0.35270863,  0.76655629,
        0.81884762])

In [12]:
spec_alt = sps.get_sed(sps_model, 'lsst')

In [13]:
sps_model.wavelengths

array([9.100e+01, 9.400e+01, 9.600e+01, ..., 9.817e+07, 9.908e+07,
       1.000e+08])

In [14]:
spec_alt[1]- sps_model.wavelengths*(1+sps_parameters[0][0])

array([-1.39143827e+02, -1.43730986e+02, -1.46789092e+02, ...,
       -1.50107137e+08, -1.51498575e+08, -1.52905304e+08])